# Deploying MultiModal Models with LMI Deep Learning Containers

In this tutorial, you will use the LMI DLC available on SageMaker to host and serve inference for a MultiModal model. We will be using the [Llava-v1.6](https://huggingface.co/llava-hf/llava-v1.6-mistral-7b-hf) model available on the HuggingFace Hub.

Please make sure that you have an IAM role with SageMaker access enabled before proceeding with this example. 

For a list of supported multimodal models in LMI, please see the documentation [here]()

## Step 1: Install Notebook Python Dependencies

In [ ]:
%pip install sagemaker --upgrade --quiet

## Step 2: Build the LMI container

In [ ]:
!git clone --branch v0.29.0 https://github.com/deepjavalibrary/djl-serving.git

In [ ]:
%%writefile djl-serving/serving/docker/llava.py

from typing import Iterable, List, Literal, Optional, Tuple, TypedDict

import torch
import torch.nn as nn
from transformers import CLIPVisionConfig, LlavaConfig

from vllm.attention import AttentionMetadata
from vllm.config import CacheConfig, MultiModalConfig
from vllm.inputs import INPUT_REGISTRY, InputContext, LLMInputs
from vllm.model_executor.layers.activation import get_act_fn
from vllm.model_executor.layers.logits_processor import LogitsProcessor
from vllm.model_executor.layers.quantization.base_config import (
    QuantizationConfig)
from vllm.model_executor.layers.sampler import Sampler
from vllm.model_executor.layers.vocab_parallel_embedding import ParallelLMHead
from vllm.model_executor.model_loader.weight_utils import default_weight_loader
from vllm.model_executor.models.clip import CLIPVisionModel
from vllm.model_executor.models.llama import LlamaModel
from vllm.model_executor.models.qwen2 import Qwen2Model
from vllm.model_executor.sampling_metadata import SamplingMetadata
from vllm.multimodal import MULTIMODAL_REGISTRY
from vllm.sequence import IntermediateTensors, SamplerOutput

from .clip import (dummy_image_for_clip, dummy_seq_data_for_clip,
                   get_max_clip_image_tokens, input_processor_for_clip)
from .interfaces import SupportsVision
from .utils import merge_vision_embeddings

_KEYS_TO_MODIFY_MAPPING = {
    "language_model.lm_head": "lm_head",
    "language_model.model": "language_model",
}

def new_qwen2_forward(
        self,
        input_ids: torch.Tensor,
        positions: torch.Tensor,
        kv_caches: List[torch.Tensor],
        attn_metadata: AttentionMetadata,
        inputs_embeds: Optional[torch.Tensor] = None,
    ) -> torch.Tensor:
        if inputs_embeds is not None:
            hidden_states = inputs_embeds
        else:
            hidden_states = self.get_input_embeddings(input_ids)
        residual = None
        for i in range(len(self.layers)):
            layer = self.layers[i]
            hidden_states, residual = layer(
                positions,
                hidden_states,
                kv_caches[i],
                attn_metadata,
                residual,
            )
        hidden_states, _ = self.norm(hidden_states, residual)
        return hidden_states

def get_input_embeddings(self, input_ids: torch.Tensor) -> torch.Tensor:
        return self.embed_tokens(input_ids)

Qwen2Model.forward = (
    new_qwen2_forward
)
Qwen2Model.get_input_embeddings = (
    get_input_embeddings
)

# TODO(xwjiang): Run benchmark and decide if TP.
class LlavaMultiModalProjector(nn.Module):

    def __init__(self, vision_hidden_size: int, text_hidden_size: int,
                 projector_hidden_act: str):
        super().__init__()

        self.linear_1 = nn.Linear(vision_hidden_size,
                                  text_hidden_size,
                                  bias=True)
        self.act = get_act_fn(projector_hidden_act)
        self.linear_2 = nn.Linear(text_hidden_size,
                                  text_hidden_size,
                                  bias=True)

    def forward(self, image_features: torch.Tensor) -> torch.Tensor:
        hidden_states = self.linear_1(image_features)
        hidden_states = self.act(hidden_states)
        hidden_states = self.linear_2(hidden_states)
        return hidden_states


class LlavaImagePixelInputs(TypedDict):
    type: Literal["pixel_values"]
    data: torch.Tensor
    """Shape: `(batch_size, num_channels, height, width)`"""


LlavaImageInputs = LlavaImagePixelInputs


def get_max_llava_image_tokens(ctx: InputContext):
    hf_config = ctx.get_hf_config(LlavaConfig)
    vision_config = hf_config.vision_config

    if isinstance(vision_config, CLIPVisionConfig):
        return get_max_clip_image_tokens(vision_config)

    msg = f"Unsupported vision config: {type(vision_config)}"
    raise NotImplementedError(msg)


def dummy_data_for_llava(ctx: InputContext, seq_len: int):
    hf_config = ctx.get_hf_config(LlavaConfig)
    vision_config = hf_config.vision_config

    if isinstance(vision_config, CLIPVisionConfig):
        seq_data = dummy_seq_data_for_clip(
            vision_config,
            seq_len,
            image_token_id=hf_config.image_token_index,
        )

        mm_data = dummy_image_for_clip(vision_config)
        return seq_data, mm_data

    msg = f"Unsupported vision config: {type(vision_config)}"
    raise NotImplementedError(msg)


def input_processor_for_llava(ctx: InputContext, llm_inputs: LLMInputs):
    multi_modal_data = llm_inputs.get("multi_modal_data")
    if multi_modal_data is None or "image" not in multi_modal_data:
        return llm_inputs

    model_config = ctx.model_config
    hf_config = ctx.get_hf_config(LlavaConfig)
    vision_config = hf_config.vision_config

    if isinstance(vision_config, CLIPVisionConfig):
        return input_processor_for_clip(
            model_config,
            vision_config,
            llm_inputs,
            image_token_id=hf_config.image_token_index,
        )

    msg = f"Unsupported vision config: {type(vision_config)}"
    raise NotImplementedError(msg)

@MULTIMODAL_REGISTRY.register_image_input_mapper()
@MULTIMODAL_REGISTRY.register_max_image_tokens(get_max_llava_image_tokens)
@INPUT_REGISTRY.register_dummy_data(dummy_data_for_llava)
@INPUT_REGISTRY.register_input_processor(input_processor_for_llava)
class LlavaForConditionalGeneration(nn.Module, SupportsVision):

    def __init__(self,
                 config: LlavaConfig,
                 multimodal_config: MultiModalConfig,
                 cache_config: Optional[CacheConfig] = None,
                 quant_config: Optional[QuantizationConfig] = None) -> None:
        super().__init__()

        self.config = config
        self.multimodal_config = multimodal_config

        # Initialize the vision tower only up to the required feature layer
        vision_feature_layer = config.vision_feature_layer
        if vision_feature_layer < 0:
            num_hidden_layers = config.vision_config.num_hidden_layers \
                + vision_feature_layer + 1
        else:
            num_hidden_layers = vision_feature_layer + 1

        # TODO: Optionally initializes this for supporting embeddings.
        self.vision_tower = CLIPVisionModel(
            config.vision_config, num_hidden_layers_override=num_hidden_layers)
        self.multi_modal_projector = LlavaMultiModalProjector(
            vision_hidden_size=config.vision_config.hidden_size,
            text_hidden_size=config.text_config.hidden_size,
            projector_hidden_act=config.projector_hidden_act)

        self.quant_config = quant_config
        ##### Change the original code to support qwen2 #####
        if config.text_config.model_type == "llama":
            self.language_model = LlamaModel(config.text_config, cache_config,
                                         quant_config)
        elif config.text_config.model_type == "qwen2":
            self.language_model = Qwen2Model(config.text_config, cache_config,
                                         quant_config)
            self.language_model.org_vocab_size = config.text_config.vocab_size
        else:
            raise ValueError(f"{config.text_config.model_type} is not supported by llava yet!!!")
        #####################################################
        self.unpadded_vocab_size = config.text_config.vocab_size
        self.lm_head = ParallelLMHead(
            self.unpadded_vocab_size,
            config.text_config.hidden_size,
            org_num_embeddings=self.language_model.org_vocab_size,
            quant_config=quant_config)
        logit_scale = getattr(config, "logit_scale", 1.0)
        self.logits_processor = LogitsProcessor(self.unpadded_vocab_size,
                                                config.text_config.vocab_size,
                                                logit_scale)
        self.sampler = Sampler()

    def _validate_pixel_values(self, data: torch.Tensor) -> torch.Tensor:
        h = w = self.config.vision_config.image_size
        expected_dims = (3, h, w)
        actual_dims = tuple(data.shape[1:])

        if actual_dims != expected_dims:
            expected_expr = ("batch_size", *map(str, expected_dims))
            raise ValueError(
                f"The expected shape of pixel values is {expected_expr}. "
                f"You supplied {tuple(data.shape)}.")

        return data

    def _parse_and_validate_image_input(
            self, **kwargs: object) -> Optional[LlavaImageInputs]:
        pixel_values = kwargs.pop("pixel_values", None)

        if pixel_values is None:
            return None

        if not isinstance(pixel_values, torch.Tensor):
            raise ValueError("Incorrect type of pixel values. "
                             f"Got type: {type(pixel_values)}")

        return LlavaImagePixelInputs(
            type="pixel_values",
            data=self._validate_pixel_values(pixel_values),
        )

    def _select_image_features(self, image_features: torch.Tensor, *,
                               strategy: str) -> torch.Tensor:
        # Copied from https://github.com/huggingface/transformers/blob/39c3c0a72af6fbda5614dde02ff236069bb79827/src/transformers/models/llava/modeling_llava.py#L421  # noqa
        if strategy == "default":
            return image_features[:, 1:]
        elif strategy == "full":
            return image_features

        raise ValueError(f"Unexpected select feature strategy: {strategy}")

    def _image_pixels_to_features(self, vision_tower: CLIPVisionModel,
                                  pixel_values: torch.Tensor) -> torch.Tensor:

        # NOTE: we skip the step to select the vision feature layer since
        # this is already done inside the vision tower
        image_features = vision_tower(pixel_values)

        return self._select_image_features(
            image_features,
            strategy=self.config.vision_feature_select_strategy,
        )

    def _process_image_pixels(self,
                              inputs: LlavaImagePixelInputs) -> torch.Tensor:
        assert self.vision_tower is not None

        pixel_values = inputs["data"]

        return self._image_pixels_to_features(self.vision_tower, pixel_values)

    def _process_image_input(self,
                             image_input: LlavaImageInputs) -> torch.Tensor:
        assert self.vision_tower is not None
        image_features = self._process_image_pixels(image_input)
        return self.multi_modal_projector(image_features)

    def forward(
        self,
        input_ids: torch.Tensor,
        positions: torch.Tensor,
        kv_caches: List[torch.Tensor],
        attn_metadata: AttentionMetadata,
        intermediate_tensors: Optional[IntermediateTensors] = None,
        **kwargs: object,
    ) -> SamplerOutput:
        """Run forward pass for LLaVA-1.5.

        One key thing to understand is the `input_ids` already accounts for the
        positions of the to-be-inserted image embeddings.

        Concretely, consider a text prompt:
        `"USER: <image>\\nWhat's the content of the image?\\nASSISTANT:"`.

        Tokenizer outputs:
        `[1, 3148, 1001, 29901, 29871, 32000, 29871, 13, 5618, 29915, 29879,
        278, 2793, 310, 278, 1967, 29973, 13, 22933, 9047, 13566, 29901]`.

        To reserve space in KV cache, we have to insert placeholder tokens
        before they are inputted to the model, so the input processor prepends 
        additional image tokens (denoted as `32000`), resulting in:
        `[1, 3148, 1001, 29901, 29871, 32000, ..., 32000, 29871, 13, 5618,
        29915, 29879, 278, 2793, 310, 278, 1967, 29973, 13, 22933, 9047, 13566,
        29901]`.

        We insert 575 tokens so that including the original image token in the
        input, there are a total of 576 (24 * 24) image tokens, which
        corresponds to the number of image tokens inputted to the language
        model, i.e. the number of image tokens outputted by the visual encoder.

        This way, the `positions` and `attn_metadata` are consistent
        with the `input_ids`.

        Args:
            input_ids: Flattened (concatenated) input_ids corresponding to a
                batch.
            pixel_values: The pixels in each input image.
        
        See also:
            :class:`LlavaImageInputs`
        """
        image_input = self._parse_and_validate_image_input(**kwargs)

        if image_input is not None:
            vision_embeddings = self._process_image_input(image_input)
            inputs_embeds = self.language_model.get_input_embeddings(input_ids)

            inputs_embeds = merge_vision_embeddings(
                input_ids, inputs_embeds, vision_embeddings,
                self.config.image_token_index)

            input_ids = None
        else:
            inputs_embeds = None


        if self.config.text_config.model_type == "llama":
            hidden_states = self.language_model(input_ids,
                                            positions,
                                            kv_caches,
                                            attn_metadata,
                                            None,
                                            inputs_embeds=inputs_embeds)
        elif self.config.text_config.model_type == "qwen2":
            hidden_states = self.language_model(input_ids,
                                            positions,
                                            kv_caches,
                                            attn_metadata,
                                            inputs_embeds=inputs_embeds)
        else:
            raise ValueError(f"{config.text_config.model_type} is not supported by llava yet!!!")

        return hidden_states

    def compute_logits(self, hidden_states: torch.Tensor,
                       sampling_metadata: SamplingMetadata) -> torch.Tensor:
        logits = self.logits_processor(self.lm_head, hidden_states,
                                       sampling_metadata)
        return logits

    def sample(
        self,
        logits: torch.Tensor,
        sampling_metadata: SamplingMetadata,
    ) -> Optional[SamplerOutput]:
        next_tokens = self.sampler(logits, sampling_metadata)
        return next_tokens

    def load_weights(self, weights: Iterable[Tuple[str, torch.Tensor]]):
        # only doing this for language model part for now.
        stacked_params_mapping = [
            # (param_name, shard_name, shard_id)
            ("qkv_proj", "q_proj", "q"),
            ("qkv_proj", "k_proj", "k"),
            ("qkv_proj", "v_proj", "v"),
            ("gate_up_proj", "gate_proj", 0),
            ("gate_up_proj", "up_proj", 1),
        ]
        params_dict = dict(self.named_parameters())
        for name, loaded_weight in weights:
            if "rotary_emb.inv_freq" in name:
                continue
            # post_layernorm is not needed in CLIPVisionModel
            if "vision_model.post_layernorm" in name:
                continue
            for key_to_modify, new_key in _KEYS_TO_MODIFY_MAPPING.items():
                if key_to_modify in name:
                    name = name.replace(key_to_modify, new_key)
            use_default_weight_loading = False
            if "vision" in name:
                if self.vision_tower is not None:
                    # We only do sharding for language model and
                    # not vision model for now.
                    use_default_weight_loading = True
            else:
                for (param_name, weight_name,
                     shard_id) in stacked_params_mapping:
                    if weight_name not in name:
                        continue
                    param = params_dict[name.replace(weight_name, param_name)]
                    weight_loader = param.weight_loader
                    weight_loader(param, loaded_weight, shard_id)
                    break
                else:
                    use_default_weight_loading = True
            if use_default_weight_loading and name in params_dict:
                param = params_dict[name]
                weight_loader = getattr(param, "weight_loader",
                                        default_weight_loader)
                weight_loader(param, loaded_weight)

In [ ]:
%%writefile djl-serving/serving/docker/lmi.Dockerfile

# -*- mode: dockerfile -*-
# Copyright 2022 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License"). You may not use this file
# except in compliance with the License. A copy of the License is located at
#
# http://aws.amazon.com/apache2.0/
#
# or in the "LICENSE.txt" file accompanying this file. This file is distributed on an "AS IS"
# BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, express or implied. See the License for
# the specific language governing permissions and limitations under the License.
ARG version=12.4.1-cudnn-devel-ubuntu22.04
FROM nvidia/cuda:$version
ARG cuda_version=cu124
ARG djl_version=0.29.0~SNAPSHOT
# Base Deps
ARG python_version=3.10
ARG torch_version=2.3.1
ARG torch_vision_version=0.18.1
ARG onnx_version=1.18.0
ARG onnxruntime_wheel="https://publish.djl.ai/onnxruntime/1.18.0/onnxruntime_gpu-1.18.0-cp310-cp310-linux_x86_64.whl"
ARG pydantic_version=2.8.2
ARG djl_converter_wheel="https://publish.djl.ai/djl_converter/djl_converter-0.28.0-py3-none-any.whl"
# HF Deps
ARG protobuf_version=3.20.3
ARG transformers_version=4.43.2
ARG accelerate_version=0.32.1
ARG bitsandbytes_version=0.43.1
ARG optimum_version=1.21.2
ARG auto_gptq_version=0.7.1
ARG datasets_version=2.20.0
ARG autoawq_version=0.2.5
ARG tokenizers_version=0.19.1
# LMI-Dist Deps
ARG vllm_wheel="https://publish.djl.ai/vllm/cu124-pt231/vllm-0.5.3.post1%2Bcu124-cp310-cp310-linux_x86_64.whl"
ARG flash_attn_2_wheel="https://github.com/vllm-project/flash-attention/releases/download/v2.5.9.post1/vllm_flash_attn-2.5.9.post1-cp310-cp310-manylinux1_x86_64.whl"
ARG flash_infer_wheel="https://github.com/flashinfer-ai/flashinfer/releases/download/v0.0.9/flashinfer-0.0.9+cu121torch2.3-cp310-cp310-linux_x86_64.whl"
# %2B is the url escape for the '+' character
ARG lmi_dist_wheel="https://publish.djl.ai/lmi_dist/lmi_dist-11.0.0-py3-none-any.whl"
ARG seq_scheduler_wheel="https://publish.djl.ai/seq_scheduler/seq_scheduler-0.1.0-py3-none-any.whl"
ARG peft_version=0.11.1

EXPOSE 8080

COPY dockerd-entrypoint-with-cuda-compat.sh /usr/local/bin/dockerd-entrypoint.sh
RUN chmod +x /usr/local/bin/dockerd-entrypoint.sh
WORKDIR /opt/djl
ENV JAVA_HOME=/usr/lib/jvm/java-17-amazon-corretto
# ENV NO_OMP_NUM_THREADS=true
ENV JAVA_OPTS="-Xmx1g -Xms1g -XX:+ExitOnOutOfMemoryError -Dai.djl.util.cuda.fork=true"
ENV MODEL_SERVER_HOME=/opt/djl
ENV MODEL_LOADING_TIMEOUT=1200
ENV PREDICT_TIMEOUT=240
ENV DJL_CACHE_DIR=/tmp/.djl.ai
ENV PYTORCH_LIBRARY_PATH=/usr/local/lib/python3.10/dist-packages/torch/lib
ENV PYTORCH_PRECXX11=true
ENV PYTORCH_VERSION=${torch_version}
ENV PYTORCH_FLAVOR=cu121-precxx11
ENV VLLM_NO_USAGE_STATS=1
ENV VLLM_WORKER_MULTIPROC_METHOD=spawn


ENV HF_HOME=/tmp/.cache/huggingface
ENV PYTORCH_KERNEL_CACHE_PATH=/tmp/.cache
ENV BITSANDBYTES_NOWELCOME=1
ENV USE_AICCL_BACKEND=true
ENV HF_HUB_ENABLE_HF_TRANSFER=1
ENV SAFETENSORS_FAST_GPU=1
ENV TORCH_NCCL_BLOCKING_WAIT=0
ENV NCCL_ASYNC_ERROR_HANDLING=1
ENV TORCH_NCCL_AVOID_RECORD_STREAMS=1
ENV SERVING_FEATURES=vllm,lmi-dist

ENTRYPOINT ["/usr/local/bin/dockerd-entrypoint.sh"]
CMD ["serve"]

COPY scripts scripts/
RUN mkdir -p /opt/djl/conf \
    && mkdir -p /opt/djl/deps \
    && mkdir -p /opt/djl/partition \
    && mkdir -p /opt/ml/model
COPY config.properties /opt/djl/conf/config.properties
COPY partition /opt/djl/partition

#COPY distribution[s]/ ./
#RUN mv *.deb djl-serving_all.deb || true

RUN apt-get update && DEBIAN_FRONTEND=noninteractive apt-get install -yq libaio-dev libopenmpi-dev g++ \
    && scripts/install_openssh.sh \
    && scripts/install_djl_serving.sh $djl_version \
    && scripts/install_djl_serving.sh $djl_version ${torch_version} \
    && djl-serving -i ai.djl.onnxruntime:onnxruntime-engine:$djl_version \
    && djl-serving -i com.microsoft.onnxruntime:onnxruntime_gpu:$onnx_version \
    && scripts/install_python.sh ${python_version} \
    && scripts/install_s5cmd.sh x64 \
    && mkdir -p /opt/djl/bin && cp scripts/telemetry.sh /opt/djl/bin \
    && echo "${djl_version} lmi" > /opt/djl/bin/telemetry \
    && pip3 cache purge \
    && apt-get clean -y && rm -rf /var/lib/apt/lists/*

RUN pip3 install torch==${torch_version} torchvision==${torch_vision_version} --extra-index-url https://download.pytorch.org/whl/cu121 \
    ${seq_scheduler_wheel} peft==${peft_version} protobuf==${protobuf_version} \
    transformers==${transformers_version} hf-transfer zstandard datasets==${datasets_version} \
    mpi4py sentencepiece tiktoken blobfile einops accelerate==${accelerate_version} bitsandbytes==${bitsandbytes_version} \
    auto-gptq==${auto_gptq_version} pandas pyarrow jinja2 retrying \
    opencv-contrib-python-headless safetensors scipy onnx sentence_transformers ${onnxruntime_wheel} autoawq==${autoawq_version} \
    tokenizers==${tokenizers_version} pydantic==${pydantic_version} \
    # TODO: installing optimum here due to version conflict.
    && pip3 install ${djl_converter_wheel} optimum==${optimum_version} --no-deps \
    && git clone https://github.com/neuralmagic/AutoFP8.git && cd AutoFP8 && git reset --hard 4b2092c && pip3 install . && cd .. && rm -rf AutoFP8 \
    && pip3 cache purge

RUN pip3 install ${flash_attn_2_wheel} ${lmi_dist_wheel} ${vllm_wheel} ${flash_infer_wheel} \
    && pip3 cache purge
COPY llava.py /usr/local/lib/python3.10/dist-packages/vllm/model_executor/models/llava.py

# Add CUDA-Compat
RUN apt-get update && apt-get install -y cuda-compat-12-4 && apt-get clean -y && rm -rf /var/lib/apt/lists/*

RUN scripts/patch_oss_dlc.sh python \
    && scripts/security_patch.sh lmi \
    && useradd -m -d /home/djl djl \
    && chown -R djl:djl /opt/djl \
    && rm -rf scripts \
    && pip3 cache purge \
    && apt-get clean -y && rm -rf /var/lib/apt/lists/*

LABEL maintainer="djl-dev@amazon.com"
LABEL dlc_major_version="1"
LABEL com.amazonaws.ml.engines.sagemaker.dlc.framework.djl.lmi="true"
LABEL com.amazonaws.ml.engines.sagemaker.dlc.framework.djl.v0-29-0.lmi="true"
LABEL com.amazonaws.sagemaker.capabilities.multi-models="true"
LABEL com.amazonaws.sagemaker.capabilities.accept-bind-to-port="true"
LABEL djl-version=$djl_version
LABEL cuda-version=$cuda_version
# To use the 535 CUDA driver, CUDA 12.1 can work on this one too
LABEL com.amazonaws.sagemaker.inference.cuda.verified_versions=12.2

In [ ]:
%%writefile sm_build.sh

#!/bin/bash

REPO_NAME=${1:-custom-aws-lmi-ecr}
DJL_VERSION=${3:-0.29.0}
ACCOUNT_ID=$(aws sts get-caller-identity --query Account --output text)
REGION=$(aws configure get region)

echo "REPO_NAME: ${REPO_NAME}"
echo "REGION: ${REGION}"
echo "ACCOUNT_ID: ${ACCOUNT_ID}"

aws ecr get-login-password --region $REGION | docker login --username AWS --password-stdin $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com

aws ecr describe-repositories --repository-names ${REPO_NAME} --region $REGION
if [ $? -ne 0 ]
then
    echo "Creating ECR repository: ${REPO_NAME}"
    aws ecr create-repository --repository-name $REPO_NAME --region $REGION
fi

cd djl-serving/serving/docker

docker build --build-arg djl_version=${DJL_VERSION} -f lmi.Dockerfile  -t $REPO_NAME .

docker tag $REPO_NAME:latest $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/$REPO_NAME:latest

docker push $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/$REPO_NAME:latest

echo "Container URI:"
echo "$ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/$REPO_NAME:latest"

In [ ]:
!bash sm_build.sh

## Step 3: Leverage the SageMaker PythonSDK to deploy your endpoint

In [ ]:
import sagemaker
from sagemaker.djl_inference import DJLModel
from sagemaker import image_uris

role = sagemaker.get_execution_role() # iam role for the endpoint
session = sagemaker.session.Session() # sagemaker session for interacting with aws APIs

In [ ]:
import boto3

def get_aws_region():
    # Get the current AWS region from the default session
    session = boto3.session.Session()
    return session.region_name

def get_aws_account_id():
    # Get the current AWS account ID from the default session
    sts_client = boto3.client("sts")
    response = sts_client.get_caller_identity()
    return response["Account"]

REGION = get_aws_region()
ACCOUNT_ID = get_aws_account_id()
REPO_NAME="custom-aws-lmi-ecr"

In [ ]:
# Choose the customized version of LMI image directly:
image_uri = f"{ACCOUNT_ID}.dkr.ecr.{REGION}.amazonaws.com/{REPO_NAME}"

In [ ]:
model = DJLModel(
    model_id="aws-prototyping/long-llava-qwen2-7b",
    role=role,
    image_uri=image_uri,
)

In [ ]:
predictor = model.deploy(initial_instance_count=1, instance_type="ml.g5.4xlarge")

## Step 4: Make Inference Predictions

For multimodal models, LMI containers support the [OpenAI Chat Completions Schema](https://platform.openai.com/docs/guides/chat-completions). You can find specific details about LMI's implementation of this spec [here](https://docs.djl.ai/docs/serving/serving/docs/lmi/user_guides/chat_input_output_schema.html).

The OpenAI Chat Completions Schema allows two methods of specifying the image data:

* an image url (e.g. https://resources.djl.ai/images/dog_bike_car.jpg)
* base64 encoded string of the image data

If an image url is provided, the container will make a network call to fetch the image. This is ok for small applications and experimentation, but is not recommended in a production setting. If you are in a network isolated environment you must use the base64 encoded string representation.

We will demonstrate both mechanisms here.

### Getting a Test Image

You are free to use any image that you want. In this example, we'll be using the following image.

In [ ]:
%pip install Pillow

In [ ]:
import urllib.request
from PIL import Image

image_url = "https://resources.djl.ai/images/dog_bike_car.jpg"
#image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
image_path = "dog_bike_car.jpg"
# download the image locally
urllib.request.urlretrieve(image_url, image_path)

In [ ]:
img = Image.open('dog_bike_car.jpg')
img.show()

### Using the image http url directly

In [ ]:
messages = {
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What is this image about?"
                }, 
                {
                    "type": "image_url",
                    "image_url": {
                        "url": image_url
                    }
                }
            ]
        }
    ],
    "seed": 1000, 
    "max_tokens": 100,
}

In [ ]:
response = predictor.predict(messages)

In [ ]:
print(response["choices"][0]["message"]["content"])

In [ ]:
response

## Using the base64 encoded image

In [ ]:
import base64

def encode_image_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
    
encoded_image = encode_image_base64(image_path)
base64_image_url = f"data:image/jpeg;base64,{encoded_image}"

In [ ]:
messages = {
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What is this image about?"
                }, 
                {
                    "type": "image_url",
                    "image_url": {
                        "url": base64_image_url
                    }
                }
            ]
        }
    ],
     "seed": 1000, 
    "max_tokens": 100,
}

In [ ]:
response = predictor.predict(messages)

In [ ]:
print(response["choices"][0]["message"]["content"])

In [ ]:
response

## Clean Up Resources

This example demonstrates how to use the LMI container to deploy MultiModal models and serve inference requests. The 0.29.0-lmi container supports a variety of multimodal models using the OpenAI Chat Completions API spec. In the future, we plan to increase the set of multimodal architectures supported, as well as provide additional API specs that can be used to make inference requests.

In [ ]:
predictor.delete_endpoint()
model.delete_model()

In [ ]:
# delete downloaded repo, image, and script
%rm -rf djl-serving
%rm dog_bike_car.jpg
%rm sm_build.sh